In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [4]:
# Q10: Produce a list of facilities with a total revenue less than 1000.

df_rev = pd.read_sql_query(
   'SELECT name, total_revenue \
    FROM \
       (SELECT name, SUM(revenue) AS total_revenue \
        FROM \
           (SELECT b.memid, b.slots, f.name, \
                CASE WHEN b.memid = 0 THEN (b.slots * 2 * f.guestcost) \
                ELSE (b.slots * 2 * f.membercost) END AS revenue \
            FROM Bookings AS b \
            LEFT JOIN Facilities AS f \
            ON b.facid = f.facid) AS sub \
        GROUP BY name) AS sub1 \
    WHERE total_revenue < 1000', engine)

In [5]:
df_rev

,name,total_revenue
0,Pool Table,540
1,Snooker Table,480
2,Table Tennis,360


In [6]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
    
df_mem = pd.read_sql_query(
   'SELECT m1.memid, m1.firstname, m1.surname, m1.recommendedby, m2.firstname || " " || m2.surname AS recommended_member \
    FROM Members AS m1 \
    LEFT JOIN Members AS m2 \
    ON m1.recommendedby = m2.memid \
    ORDER BY m1.surname, m1.firstname', engine)

In [7]:
df_mem.sample(5)

,memid,firstname,surname,recommendedby,recommended_member
16,21,Anna,Mackenzie,1,Darren Smith
24,37,Darren,Smith,,None
4,5,Gerald,Butters,1,Darren Smith
12,35,John,Hunt,30,Millicent Purview
7,7,Nancy,Dare,4,Janice Joplette


In [8]:
# Q12: Find the facilities with their usage by member, but not guests 

df_usage = pd.read_sql_query(
   'SELECT f.name, sub.total_use \
    FROM ( \
        SELECT facid, SUM(slots) AS total_use \
        FROM Bookings \
        WHERE memid != 0 \
        GROUP BY facid) AS sub \
    LEFT JOIN Facilities AS f \
    ON sub.facid = f.facid \
    ORDER BY sub.total_use;', engine)

In [9]:
df_usage

,name,total_use
0,Massage Room 2,54
1,Squash Court,418
2,Table Tennis,794
3,Pool Table,856
4,Snooker Table,860
5,Tennis Court 2,882
6,Massage Room 1,884
7,Tennis Court 1,957
8,Badminton Court,1086


In [10]:
# Q13: Find the facilities usage by month, but not guests 

df_monthly_usage = pd.read_sql_query(
   'SELECT f.name, sub.month, sub.total_use \
    FROM( \
        SELECT facid, memid, SUM(slots) AS total_use, STRFTIME("%m", starttime) AS month \
        FROM Bookings \
        WHERE memid != 0 \
        GROUP BY facid, month) AS sub \
    LEFT JOIN Facilities AS f \
    ON sub.facid = f.facid;', engine)

In [11]:
df_monthly_usage

,name,month,total_use
0,Tennis Court 1,07,201
1,Tennis Court 1,08,339
2,Tennis Court 1,09,417
3,Tennis Court 2,07,123
4,Tennis Court 2,08,345
5,Tennis Court 2,09,414
6,Badminton Court,07,165
7,Badminton Court,08,414
8,Badminton Court,09,507
9,Table Tennis,07,98
